In [1]:
import os
import numpy as np
import sys
import socket
import pickle
import math
import os
import json
from tqdm import tqdm
import torch
import k3d

from pytorch3d.io import load_obj,save_ply
from pytorch3d.ops import sample_points_from_meshes,knn_points
from pytorch3d.structures import Pointclouds,Meshes

In [2]:
pix_path = '/scratch/fml35/datasets/pix3d_new/'
with open("/data/cornucopia/fml35/experiments/exp_024_debug/models/model_list.json",'r') as f:
        model_list = json.load(f)["models"]

for j in tqdm(range(0,1)):#len(model_list))):
    path = pix_path + model_list[j]["model"]

100%|██████████| 1/1 [00:00<00:00, 7182.03it/s]


In [3]:
verts_torch,faces_torch,_ = load_obj(path,load_textures=False)

verts = verts_torch.numpy()
faces = faces_torch[0].numpy()

print(np.max(verts,axis=0))
print(np.min(verts,axis=0))
print('verts',verts.shape)

[0.28645244 0.14048965 0.38497746]
[-0.28645244 -0.14048965 -0.38497746]
verts (137, 3)


In [4]:
n_points = 50000
n_nn = 5
angle_threshold_degree = 20


mesh = Meshes(verts=[verts_torch],faces=[faces_torch[0]])
sample_points,sample_normals = sample_points_from_meshes(mesh,n_points,return_normals=True)
sample_points = sample_points.squeeze()
sample_normals = sample_normals.squeeze()

In [5]:

# find distance between two pointclouds
# For each predicted point, find its neareast-neighbor GT point
knn_pred = knn_points(sample_points.unsqueeze(0),sample_points.unsqueeze(0), K=n_nn)

sample_normals_unsq = sample_normals.unsqueeze(1)
sample_normals_tiled = sample_normals_unsq.tile(1,n_nn,1)

nn_normals = torch.zeros(n_points,n_nn,3)
for i in tqdm(range(n_points)):
    n = sample_normals[knn_pred.idx[0,i]]
    nn_normals[i] = n
    
normal_dot = torch.sum(sample_normals_tiled*nn_normals,dim=2)
angles = torch.arccos(normal_dot) * 180 / np.pi
angles = torch.min(angles,torch.abs(180. - angles))
# TODO: think if better 180 or 360
# angles = torch.min(angles,torch.abs(360. - angles))
max_angles,_ = torch.max(angles,dim=1)
labels = (max_angles > angle_threshold_degree)
labels = labels.numpy()
print(labels.shape)



100%|██████████| 50000/50000 [00:00<00:00, 59505.14it/s]


(50000,)


In [6]:

plot = k3d.plot()

# plot += k3d.points(sample_points.numpy()[~labels],point_size=0.01,color=(255))
plot += k3d.points(sample_points.numpy()[labels],point_size=0.01,color=(0 + 256*255))
plot.display()

Output()

In [7]:
# maybe just do an optimisation, only free parameter is number of lines optimise by adjusting for every line what are the start coordinates and end coordinates
# minimise the distance of all points to the nearest line, constrain line endpoints to lie within reasonable volume

def dist_point_line(lp,p):
    print(lp.shape)
    print(p.shape)
    # lp: N_lines x 2 (x1,y1,x2,y2) and p: N_points x 2
    N_l = lp.shape[0]
    N_p = p.shape[0]

    x1 = lp[:,0].repeat(N_p)
    y1 = lp[:,1].repeat(N_p)
    x2 = lp[:,2].repeat(N_p)
    y2 = lp[:,3].repeat(N_p)
    x0 = p[:,0].repeat_interleave(N_l)
    y0 = p[:,1].repeat_interleave(N_l)


    px = x2-x1
    py = y2-y1

    norm = px*px + py*py

    u =  ((x0 - x1) * px + (y0 - y1) * py) / norm

    u = torch.clip(u,min=0,max=1)

    x = x1 + u * px
    y = y1 + u * py

    dx = x - x0
    dy = y - y0

    dist = (dx*dx + dy*dy)**.5

    # dist = torch.abs((x2 - x1) * (y1 - y0) - (x1 - x0) * (y2 - y1)) / torch.sqrt((x2-x1)**2 + (y2 -y1)**2)
    dist = dist.reshape(N_p,N_l)
    dist,_ = torch.min(dist,dim=1)

    return dist



In [8]:
def dist_point_line(lp,p):
    # lp: N_lines x 2 (x1,y1,x2,y2) and p: N_points x 2
    N_l = lp.shape[0]
    N_p = p.shape[0]

    x1 = lp[:,0].repeat(N_p)
    y1 = lp[:,1].repeat(N_p)
    z1 = lp[:,2].repeat(N_p)
    x2 = lp[:,3].repeat(N_p)
    y2 = lp[:,4].repeat(N_p)
    z2 = lp[:,5].repeat(N_p)
    x0 = p[:,0].repeat_interleave(N_l)
    y0 = p[:,1].repeat_interleave(N_l)
    z0 = p[:,2].repeat_interleave(N_l)


    px = x2-x1
    py = y2-y1
    pz = z2-z1

    norm = px*px + py*py + pz*pz

    # print('norm',norm)

    u =  ((x0 - x1) * px + (y0 - y1) * py + (z0 - z1) * pz) / norm

    u = torch.clip(u,min=0,max=1)

    # print('u',u)

    x = x1 + u * px
    y = y1 + u * py
    z = z1 + u * pz

    dx = x - x0
    dy = y - y0
    dz = z - z0



    dist_non_flattened = (dx*dx + dy*dy + dz*dz)#**.5

    # print('dist_non_flatten',dist_non_flattened)
    # dist = torch.abs(dx) + torch.abs(dy) + torch.abs(dz)

    # dist = torch.abs((x2 - x1) * (y1 - y0) - (x1 - x0) * (y2 - y1)) / torch.sqrt((x2-x1)**2 + (y2 -y1)**2)
    dist_non_flattened = dist_non_flattened.reshape(N_p,N_l)
    dist,_ = torch.min(dist_non_flattened,dim=1)

    return dist,dist_non_flattened

In [9]:
lp = torch.Tensor([[0,0,0,0,1,0],[1,0,0,1,1,0]])
p = torch.Tensor([[0,0,0],[0.7,0.5,0],[0.5,1,0],[0.5,2,0]])
print(dist_point_line(lp,p))

(tensor([0.0000, 0.0900, 0.2500, 1.2500]), tensor([[0.0000, 1.0000],
        [0.4900, 0.0900],
        [0.2500, 0.2500],
        [1.2500, 1.2500]]))


In [15]:
def create_plot(points_optimise,line_points,mask,distances):
    plot = k3d.plot()

    plot += k3d.points(points_optimise,point_size=0.03,color=(0 + 256*255))
    plot += k3d.points(points_optimise[distances > 0.00125],point_size=0.03,color=(255))
    for i in range(n_lines):
        if mask is None or mask[i] == True: 
            plot += k3d.line([line_points[i,:3].numpy(), line_points[i,3:6].numpy()], shader='mesh', width=.003, color=0xff0000)
    plot.display()

In [11]:
n_lines = 10
n_steps = 1000


# maybe better initialisation, sample points from mesh 
# maybe should do Hough transform and then sample pairs of points and check for clusters (which correspond to lines)
# start = torch.rand((n_lines,6)) - 0.5

# points_optimise = torch.Tensor([[0,0,0],[1,1,1],[0,0.2,0.4],[0,0.23,0.4]])
points_optimise = torch.from_numpy(sample_points.numpy()[labels])

start_ids = torch.randint(low=0,high=points_optimise.shape[0],size=(n_lines,2))
# print(start_ids)
# print(start_ids.shape)
start = points_optimise[start_ids]
start[:,0,:] = torch.rand((start.shape[0],3)) - 0.5
start = start.view(-1,6)

# print(start)
# print(start_ids)

In [17]:
# print(start.shape)
line_points = start.clone()
line_points.requires_grad = True
# points_optimise = torch.Tensor([[0,0,0],[1,1,1],[0,0.2,0.4],[0,0.23,0.4]])

# optimizer = torch.optim.SGD([line_points], lr=1e2,momentum=0.0,dampening=0, weight_decay=0, nesterov=False)
optimizer = torch.optim.Adam([line_points], lr=1e-1)
print(points_optimise.shape)
for step in tqdm(range(n_steps)):
        
    optimizer.zero_grad()

    distances,non_flat = dist_point_line(line_points,points_optimise)
    # print('distances.shape',distances[:5])
    # print('non_flat.shape',non_flat[:5])
    # print('distances.tile(1,n_lines)',distances.tile(1,n_lines).shape)
    mask = distances.unsqueeze(1).tile(1,n_lines) == non_flat
    mask_all_points = mask.any(dim=0)
    # print(mask)
    # print(mask.shape)
    # print(mask_all_points)

    if (step ) % 20 == 0:
        # print('line_points',line_points)
        # print('points_optimise',points_optimise[:3])
        # print('non flat',non_flat[:6])
        print('dists ',distances[:])
        # print('loss',loss)
        create_plot(points_optimise,line_points.detach(),mask_all_points,distances)
        print(torch.sum(mask_all_points))
        # print(mask)
        # print(mask_all_points.shape)
        # print(mask_all_points)
        pass
    

    loss = torch.mean(distances)
    loss.backward()
    optimizer.step()

    # line_points = line_points.detach()

    # for i in range(n_lines):
    #     if not mask_all_points[i]:
    #         line_points[i] = torch.rand((6)) - 0.5
        
    # line_points.requires_grad = True

# create_plot(points_optimise,line_points.detach(),None)
create_plot(points_optimise,line_points.detach(),mask_all_points)



torch.Size([7535, 3])


  0%|          | 0/1000 [00:00<?, ?it/s]

dists  tensor([0.0020, 0.0114, 0.0005,  ..., 0.0158, 0.0014, 0.0878],
       grad_fn=<SliceBackward0>)


Output()

tensor(10)


...

 32%|███▏      | 320/1000 [19:22<10:40,  1.06it/s]

dists  tensor([0.0006, 0.0005, 0.0007,  ..., 0.0051, 0.0002, 0.0046],
       grad_fn=<SliceBackward0>)


Output()

tensor(7)


 33%|███▎      | 327/1000 [19:25<05:27,  2.06it/s]